In [1]:
import os
import subprocess

# Paths and configurations
pdb_files = [
    "/home/hp/nayanika/github/GPX6/prep_structures/mousecys/D25N.pdb",
    "/home/hp/nayanika/github/GPX6/prep_structures/mousecys/G27E.pdb",
    # Add more PDB files as needed
]
base_scr_dir = "/home/hp/results/mousecys"
run_dir = "/home/hp/nayanika/github/GPX6/cluster_scripts"
genrelax_dir = "/home/hp/nayanika/github/GPX6/input"
topology_dir = "/home/hp/nayanika/github/GPX6/prep_structures/mousecys"
fep_dir = "/home/hp/nayanika/github/GPX6/input/fep"  # Correct FEP directory path

# Script names
run_qdyn_file = os.path.join(run_dir, "run_qdyn_5.sh")
run_replica_file = os.path.join(run_dir, "run_replica.sh")

# Ensure base directory exists
os.makedirs(base_scr_dir, exist_ok=True)

# Process each PDB file
for pdb_file in pdb_files:
    # Extract the system name from the PDB file
    system_name = os.path.basename(pdb_file).replace(".pdb", "")
    system_dir = os.path.join(base_scr_dir, system_name)
    os.makedirs(system_dir, exist_ok=True)

    # Define the topology file path
    topology_file = os.path.join(topology_dir, f"{system_name}.top")
    
    # Check if the topology file exists
    if not os.path.isfile(topology_file):
        print(f"No topology file found for {system_name} at {topology_file}")
        continue

    # Step 1: Run q_genrelax.py for the initial processing
    command = [
        "q_genrelax.py", 
        os.path.join(genrelax_dir, "genrelax.proc"),
        "--top", topology_file,
        "--pdb", pdb_file,
        "--fep", os.path.join(fep_dir, "GPX6_wtmousecys.fep"),
        "--outdir", os.path.join(system_dir, "minim"),
        "--rs", run_qdyn_file
    ]
    
    # Execute the command
    try:
        subprocess.run(command, check=True)
        print(f"Successfully processed {pdb_file} with topology {topology_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {pdb_file}: {e}")
        continue  # Skip to the next file if an error occurs

    # Step 2: Run q_genfeps.py and create replicas for each system
    minim_dir = os.path.join(system_dir, "minim")
    os.makedirs(minim_dir, exist_ok=True)

    q_genfeps_command = [
        "q_genfeps.py",
        "/home/hp/nayanika/github/GPX6/input/genfeps.proc",
        "--pdb", pdb_file,
        os.path.join(minim_dir, "relax_012.inp"),
        "relax",
        "--repeats", "100",
        "--frames", "51",
        "--fromlambda", "1.0",
        "--prefix", os.path.join(system_dir, "replica"),
        "--rs", run_qdyn_file
    ]
    
    try:
        subprocess.run(q_genfeps_command, check=True)
        print(f"Replica generation completed for {system_name}")
    except subprocess.CalledProcessError as e:
        print(f"Error in q_genfeps for {system_name}: {e}")
        continue

    # Step 3: Copy run_qdyn_5.sh and run_replica.sh to each replica directory and run replicas
    for replica_dir in sorted(os.listdir(system_dir)):
        if replica_dir.startswith("replica"):
            full_replica_dir = os.path.join(system_dir, replica_dir)
            os.makedirs(full_replica_dir, exist_ok=True)
            
            # Copy the qdyn script
            if os.path.isfile(run_qdyn_file):
                subprocess.run(["cp", run_qdyn_file, full_replica_dir])
            
            # Run the replica processing script if it exists
            if os.path.isfile(run_replica_file):
                subprocess.run(["bash", run_replica_file, full_replica_dir])
                print(f"Ran run_replica.sh in {full_replica_dir}")

print("Processing completed for all systems.")


# These placeholders will be replaced with atom indices: $1.N$, $195.O$, $49.SG$, $196.O1$, $83.OE1$, $196.O1$, $49.SG$, $49.HG1$, $196.O1$, $196.H1$, $83.OE1$, $196.H1$, $196.O1$, $49.HG1$
# Restraining to: 'topology'
# Replacing FEP file placeholders...
# Generated random seed in step 1: 437590
# Generated random seed in step 2: 690474
# Generated random seed in step 3: 163772
# Generated random seed in step 4: 472495
# Generated random seed in step 5: 845107
# Generated random seed in step 6: 604299
# Generated random seed in step 7: 545483
# Generated random seed in step 8: 291355
# Generated random seed in step 9: 665720
# Generated random seed in step 10: 892760
# Generated random seed in step 11: 926373
# Generated random seed in step 12: 685266
# Created inputs relax_001.inp - relax_012.inp
# Overridden parameters:
# 6: md/stepsize:0.1->0.5, md/bath_coupling:0.1->0.5, intervals/output:10->1000, intervals/trajectory:10->1000, intervals/temperature:10->1000
# 12: md/stepsize:0.1-